# RNNs for text generation experiment


## Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import random
import string
import tensorflow as tf
from six.moves import range
import sys
sys.path.append("/Users/gamal/git_local_repo/playground/utils")
import utils

# Load data

In [12]:
url = 'http://mattmahoney.net/dc/'
filename = utils.maybe_download(url,  "text8.zip", 31344016)
text = utils.read_data(filename)
print('number of characters: %d' % len(text))

Found and verified text8.zip
number of characters: 100000000


## split data to training and validation sets.

In [33]:
valid_size = 5000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print("training data size: %d characters. \n   example: "%train_size,
      " ".join(train_text[:80].split(" ")[:-1]))
print("validation data size: %d characters. \n   example: "%valid_size,
      " ".join(valid_text[:80].split(" ")[:-1]))

training data size: 99995000 characters. 
   example:   of one eight four eight in france proudhon s philosophy of property is complex
validation data size: 5000 characters. 
   example:   anarchism originated as a term of abuse first used against early working class


In [116]:
lowercase_chars = string.ascii_lowercase
number_chars = ["%d" %i for i in range(10)]
vocab_size = len(lowercase_chars)+len(number_chars)+3
def char_encode(char):
  """function that encodes chars to ids."""
  # consider lower case characters, space and dot.
  # ["a", .., "z"] ----> [1, ..., 26]
  # ["0", .., "9"] ----> [27, ..., 36]
  # " " ---> 37
  # "." ---> 38
  if char in lowercase_chars:
    return ord(char.lower())-ord("a")+1
  elif char in number_chars:
    return len(lowercase_chars)+int(char)+1
  elif char is " ":
    return len(lowercase_chars)+len(number_chars)+1
  elif char is ".":
    return len(lowercase_chars)+len(number_chars)+2
  else:
    # mapp all other chars and symbols to 0 id
    return 0

def char_decode(char_code):
  """function that decode from ids to chars."""
  # consider lower case characters, space and dot.
  if char_code < 0 or char_code > vocab_size-1:
    raise Exception("wrong character code")
  if char_code == 0:
    return "$" # all unknown symbols are mapped to '$'
  if char_code < len(lowercase_chars)+1:
    return chr(char_code+ord("a")-1)
  elif char_code < len(lowercase_chars)+len(number_chars)+1:
    return chr(char_code-len(lowercase_chars)+ord("0")-1)
  elif char_code < len(lowercase_chars)+len(number_chars)+2:
    return " "
  else:
    return "."
    

In [134]:
# test char_encode and char_decode
if (
  char_decode(char_encode("a")) +
  char_decode(char_encode("z")) +
  char_decode(char_encode("0")) +
  char_decode(char_encode("9")) +
  char_decode(char_encode(" ")) +
  char_decode(char_encode(".")) +
  char_decode(char_encode("#"))) == "az09 .$":
  print("encode decode characters test pass")
else:
  print("encode decode characters test fails")

encode decode characters test pass


## References
deep learning course at Udacity